In [1]:
import math

In [2]:
import pandas as pd

import numpy as np

import nltk

import re
from collections import Counter

import string

import pprint

In [3]:
from statistics import mean

In [4]:
from itertools import islice

Nb Formatting 

In [5]:
pd.options.display.max_seq_items = 2000

pd.set_option('display.max_columns', 100)

# Data setup

Open Dataset 

In [6]:
#ignore header for easier (ordinal) indexing
datasheet = pd.read_excel('CogData_FU.xlsx',header=None)

In [7]:
len(datasheet)

577

In [8]:
datasheet.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
0,Record ID,Event Name,Write a short sketch about a memory from your ...,How does technology and social media impact th...,Date of Neurobehavioral Status Exam,Notes,NIH Toolbox ID,NIH Toolbox Password,Crystallized Composite fully adjusted,Picture Vocab fully adjusted,Picture Vocab age adjusted,Picture Vocab computed,Oral Reading Recognition fully adjusted,Oral Reading Recognition age adjusted,Oral Reading Recognition computed,RAVLT Trial1,RAVLT Trial2,RAVLT Trial3,RAVLT Delayed Recall,RAVLT: Total Recognition Score,RAVLT: False Positive Score,RAVLT Percentile,Picture Sequence Memory fully adjusted,Picture Sequence Age Adjusted,Picture Sequence Memory computed score,List Sorting fully adjusted,List Sorting Age Adjusted,List Sorting raw score,Dimensional Change Card Sort Fully Adjusted,Dimensional Change Age Adjusted,Dimensional Change computed score,Dimensional Change raw score,Flanker Inhibitory Attention & Control Fully A...,Flanker Inhibitory Attention & Control age adj...,Flanker computed score,Flanker raw score,Pattern Comparison Fully Adjusted Scale Score,Pattern Comparison age adjusted,Pattern Comparison Raw Score,Odor Identification Fully Adjusted Scale Score,Odor Identification age adjusted,Odor Identification (Raw),Oral Digit Symbol Percentile,Oral Digit Symbol (Raw),Date of Short Cognitive Testing,FAS Letter Fluency,FAS Percentile,FAS Fully Adjusted,Animal Naming Test,Animal Naming Test Percentile,Animal Naming Fully Adjusted,Trail Making Test B,Trail Making Test B Percentile,Trail Making Fully Adjusted,Boston Naming,Boston Naming Percentile,Boston Naming Fully Adjusted,Neurotrack ID,Neurotrack Date,NT Mean Novelty Preference Score,Neurotrack Score (Median),1. Orientation Score,1a. Time,1b. Place,2. Registration,3. Attention and Calculation,4. Recall,5. Language,5a. Naming,5b. Repetition,5c. 3-stage Command,5d. Reading,5e. Writing,5f. Copying,Total MMSE score,Story Version,Immediate Recall,Immediate Recall Percentile,Delayed Recall,Delayed Recall Percentile,Facial Recognition A,Cued First Letter A,Multiple Choice Names A,Facial Recognition B,Cued First Letter B,Multiple Choice Names B,CogStateID,Date Conducted:,CogState: Detection,CogState: Identification,CogState: One Card Learning,CogState: One Back Speed,Cog State: One Back Accuracy,Ravens Progressive Matrices,Logical Memory Part B,East Boston Immediate Recall,Complete?
1,10,Baseline,NaN,NaN,2015-02-23 00:00:00,"RAVLT recognition, Odor ID and Trails B not ad...",85176,NaN,83.32,73.6571,83.1442,1464,100.106,109.371,2180,2,3,3,0,NaN,NaN,<25,95.04,95.98,407.08,68.03,74.62,8,82.9,88.57,5.85,30,68.51,72.43,4.88,19,67.64,70.46,28,NaN,NaN,NaN,<min,26,NaN,21,<10,NaN,7,<10,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,5,1,4,3,4,0,9,2,1,3,1,1,1,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
2,10,FU Visit 2,NaN,NaN,2015-09-28 00:00:00,Trails B incomplete,65967,NaN,129.69,104.74,107.1,1739,127.399,132.224,2578,3,3,4,0,9,16,<25,NaN,NaN,NaN,NaN,NaN,NaN,81.81,87.53,5.74,25,81.03,86.93,6.85,20,54.96,58.74,18,91.83,89.99,6,<25,35,NaN,27,20-30,NaN,10,<10,NaN,NaN,NaN,NaN,11,NaN,NaN,S_172,2015-09-28 00:00:00,0.548938,0.642857,5,2,3,3,5,0,9,2,1,3,1,1,1,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
3,13,Baseline,NaN,NaN,2015-02-24 00:00:00,Odor ID not administered,15497,NaN,140.09,122.101,126.974,2001,153.735,153.95,5220,8,10,14,12,NaN,NaN,>75,113.71,114.05,596.02,122.3,128,23,101.5,108.28,8.65,30,83.14,87.7,7.9,20,114.81,118.54,67,NaN,NaN,NaN,>75,108,NaN,70,>90,NaN,35,>90,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401201572678100,2015-04-01 

Row Indices of Memory Responses 

In [9]:
#ignore first (header) column
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index


Record IDs of Memory Respondents [Analyze all scores from each participant]

In [10]:
memory_id = list(datasheet.iloc[memory_ix][0])

Confirm Unique Response-Respondents 

In [11]:
len(memory_id)

46

In [12]:
len(set(memory_id))

46

In [13]:
datasheet.iloc[memory_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1054,2015-02-03 00:00:00,113,112,100,100,103,NaN,NaN,NaN,Complete
51,64,Baseline,I went to Paris when I was 16. I was staying ...,My husband and I are on our computers all day ...,2015-05-08 00:00:00,No Rey recognition,61681,NaN,130.68,134.57,139.464,2163,115.002,120.701,2379,7,10,11,6,NaN,NaN,>75,NaN,NaN,NaN,NaN,NaN,NaN,116.76,122.84,8.3,30,113.97,119.91,8.68,20,122.25,126.13,60,132.15,137.68,9,50-75,62,NaN,47,60-70,NaN,22,75,NaN,84,50,NaN,15,NaN,NaN,S_53,2015-05-08 00:00:00,poor quality,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61620151847168,2015-06-16 00:00:00,NaN,NaN,80,96,61,NaN,NaN,NaN,Complete
56,67,Baseline,My husband and I had always wanted to go to Pa...,"I'm on the computer all the time, writing or r...",2015-03-17 00:00:00,No RAVLT recognition and Odor ID,56478,NaN,130.99,129.869,132.736,2131,120.448,123.786,2449,7,10,12,10,NaN,NaN,>75,NaN,NaN,NaN,NaN,NaN,NaN,80.55,83.58,6.04,29,82.62,86.65,7.28,20,72,75.96,36,NaN,NaN,NaN,25-50,62,2015-03-17 00:00:00,49,70,NaN,29,>90,NaN,72,20-30,NaN,14,NaN,NaN,I_11,2015-03-17 00:00:00,0.570714,0.6,10,5,5,3,5,3,9,1,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325201574035660,2015-03-25 00:00:00,96,105,109,99,103,NaN,NaN,NaN,Complete
78,101,Baseline,I was born in Hungary. A significant memory is...,My use of social media is minimal. I only use ...,2015-03-06 00:00:00,No RAVLT rec and Odor ID,52303,NaN,123.55,117.09,121.136,1967,120.986,124.254,2455,6,9,10,7,NaN,NaN,>75,NaN,NaN,NaN,97.45,103.11,16,92.3,97.7,7.31,29,78.6,81.54,6.8,20,88.52,92.83,46,NaN,NaN,NaN,25-50,60,NaN,30,2018-10-20 00:00:00,NaN,11,<10,NaN,66,60-70,NaN,13,NaN,NaN,I_00,2015-03-06 00:00:00,0.470714,0.5,10,5,5,3,5,2,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422201573922344,2015-04-22 00:00:00,NaN,113,72,NaN,NaN,NaN,NaN,NaN,Complete
89,110,Baseline,When I was 16 years old I traveled to Europe b...,I check that on what my children are doing and...,2016-03-04 00:00:00,NaN,78039,vab,137.14,133,137.148,2164,124.176,129.631,2505,9,12,15,13,14,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,111.59,118.1,9,30,118.52,124.95,9.56,20,128.01,132.7,70,111.87,118.02,9,50-75,78,2016-03-04 00:00:00,76,>90,NaN,28,75-90,NaN,35,>90,NaN,15,NaN,NaN,I_338,2016-03-01 00:00:00,0.595952,0.595952,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,304201673920436,2016-03-04 00:00:00,106,105,112,102,103,NaN,NaN,NaN,Complete
100,135,Baseline,I have great memories of growing up in huge ho...,Technology has helped me connect to family and...,2015-04-20 00:00:00,No RAVLT rec.,59106,NaN,105.68,106.366,108.008,1767,104.435,109.664,2163,4,8,9,7,NaN,NaN,25-50,NaN,NaN,NaN,NaN,NaN,NaN,

Tech respondents

In [14]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [15]:
tech_ix

Int64Index([  6,  29,  51,  56,  78,  89, 100, 107, 118, 154, 184, 202, 217,
            226, 234, 242, 266, 277, 280, 286, 297, 303, 324, 337, 347, 352,
            359, 375, 387, 398, 411, 423, 433, 438, 445, 448, 460, 464, 470,
            474, 477, 496, 499, 502, 506, 509, 519, 522, 537, 540],
           dtype='int64')

In [16]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [17]:
len(tech_id)

50

In [18]:
#each respondent responded to one prompt
len(tech_id) == len(set(tech_id))

True

In [19]:
ix_id_series = pd.Series(tech_id,index=tech_ix)

In [20]:
ix_id_series

6       14
29      26
51      64
56      67
78     101
89     110
100    135
107    139
118    145
154    205
184    233
202    245
217    254
226    263
234    271
242    276
266    291
277    310
280    320
286    322
297    331
303    342
324    382
337    403
347    417
352    420
359    434
375    458
387    465
398    474
411    483
423    512
433    514
438    521
445    532
448    537
460    559
464    567
470    579
474    583
477    587
496    602
499    603
502    604
506    607
509    609
519    616
522    619
537    626
540    631
dtype: int64

Tech-exclusive respondents

In [21]:
#row indices of tech-respondents, mem-negligent
tech_x_ix = [tix for tix in tech_ix if tix not in memory_ix]

In [22]:
#record ids of tech-respondents who are mem-negligent
tech_x_ids = [tid for tid in tech_id if tid not in memory_id]

Mem-exclusive respondents

In [23]:
#mem respondents who are tech-negligent
[m for m in memory_id if m not in tech_id]

[]

Indices of all Respondent-rows, by record ID

In [24]:
#all rows for tech_respondents
tech_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in tech_id]

In [25]:
datasheet.iloc[tech_all_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,129.483,2018,119.487,124.812,2448,4,8,9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.73,127.93,8.87,27,113.43,119.36,8.65,20,122.25,126.13,60,77.81,80.25,5,<50,59,NaN,50,>70,NaN,16,25,NaN,90,30-40,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,702201572669916,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,135.169,2128,119.993,125.099,2457,2,7,8,3,9,1,50-75,NaN,NaN,NaN,NaN,NaN,NaN,125.63,131.41,9.22,30,116.99,122.7,8.84,20,122.25,126.13,60,112,116.35,8,50-75,69,2016-03-07 00:00:00,50,80-90,NaN,25,>90,NaN,95,60,NaN,15,NaN,NaN,I_340,2016-03-07 00:00:00,unscorable,NaN,9,4,5,3,5,3,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307201672669916,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,136.123,2138,113.024,118.634,2344,7,9,11,9,11,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,120.66,125.95,8.66,29,110.39,116.18,8.5,20,112.71,117.19,55,87.26,89.99,6,50,60,2016-08-25 00:00:00,45,70-80,NaN,16,25-50,NaN,95,50,NaN,15,NaN,NaN,I_410,2016-08-25 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,10,NaN,8,NaN,12,1,8,NaN,NaN,NaN,702201572669916,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,139.433,2163,119.738,124.89,2452,5,6,8,5,11,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,116.47,122.56,8.27,29,116.44,122.32,8.8,20,116.53,120.58,57,98.2,101.73,7,50-75,71,2017-02-17 00:00:00,55,80-90,NaN,24,75-90,NaN,103,30,NaN,15,NaN,NaN,I_481,2017-02-17 00:00:00,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,B,8,NaN,6,NaN,12,3,5,NaN,NaN,NaN,217201772669916,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,134.17,2111,105.981,113.037,2241,4,6,9,4,11,6,50-75,NaN,NaN,NaN,NaN,NaN,NaN,118.24,124.23,8.42,30,113.82,119.68,8.67,20,120.2,124.32,59,98.2,101.73,7,50-75,63,2017-09-15 00:00:00,40,40-50,NaN,18,25-50,NaN,84,50,NaN,15,NaN,NaN,I_597,2017-09-15 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,11,NaN,11,NaN,NaN,NaN,NaN,12,5,11,217201772669916,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,141.764,2189,114.399,120.015,2370,5,7,10,6,10,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,119.76,125.35,8.56,30,108.67,114.54,8.41,20,125.84,129.77,62,98.2,101.73,7,50-75,61,2018-04-16 00:00:00,51,70-80,NaN,17,25-50,NaN,113,<10,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,9,4,5,3,5,2,9,2,1,3,1,1,1,28,B,10,NaN,9,NaN,12,5,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN

In [26]:
mem_all_ix = [ix for ix in datasheet.index if datasheet.iloc[ix,0] in memory_id]

In [27]:
datasheet.iloc[mem_all_ix]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
6,14,Baseline,When I was a little child I had the same exper...,I stay in touch with family and friends throug...,2015-02-26 00:00:00,NaN,35926,qjh,123.54,126.545,131.008,2048,114.984,120.673,2378,4,3,6,2,NaN,NaN,NaN,99.13,104.48,455.07,94.85,101.4,14,106.02,112.97,7.52,30,84.37,88.37,6.97,20,97.93,102.12,47,NaN,NaN,NaN,25-50,57,NaN,46,60-70,NaN,20,50-75,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317201572669916,2015-03-17 00:00:00,102,99,103,93,116,NaN,NaN,NaN,Complete
7,14,FU Visit 2,NaN,NaN,2015-07-14 00:00:00,NaN,82858,NaN,126.32,124.077,129.483,2018,119.487,124.812,2448,4,8,9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.73,127.93,8.87,27,113.43,119.36,8.65,20,122.25,126.13,60,77.81,80.25,5,<50,59,NaN,50,>70,NaN,16,25,NaN,90,30-40,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,702201572669916,2015-07-02 00:00:00,108,99,95,91,107,NaN,NaN,NaN,Complete
8,14,FU Visit 3,NaN,NaN,2016-03-07 00:00:00,NaN,48846,evf,132.09,132.171,135.169,2128,119.993,125.099,2457,2,7,8,3,9,1,50-75,NaN,NaN,NaN,NaN,NaN,NaN,125.63,131.41,9.22,30,116.99,122.7,8.84,20,122.25,126.13,60,112,116.35,8,50-75,69,2016-03-07 00:00:00,50,80-90,NaN,25,>90,NaN,95,60,NaN,15,NaN,NaN,I_340,2016-03-07 00:00:00,unscorable,NaN,9,4,5,3,5,3,9,2,1,3,1,1,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307201672669916,2016-03-07 00:00:00,102,98,110,92,101,NaN,NaN,NaN,Complete
9,14,FU Visit 4,NaN,NaN,2016-08-25 00:00:00,NaN,94255,yua,126.02,132.98,136.123,2138,113.024,118.634,2344,7,9,11,9,11,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,120.66,125.95,8.66,29,110.39,116.18,8.5,20,112.71,117.19,55,87.26,89.99,6,50,60,2016-08-25 00:00:00,45,70-80,NaN,16,25-50,NaN,95,50,NaN,15,NaN,NaN,I_410,2016-08-25 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,10,NaN,8,NaN,12,1,8,NaN,NaN,NaN,702201572669916,2016-08-25 00:00:00,105,94,104,86,103,NaN,NaN,NaN,Complete
10,14,FU Visit 5,NaN,NaN,2017-02-17 00:00:00,NaN,45760,mur,134.39,134.534,139.433,2163,119.738,124.89,2452,5,6,8,5,11,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,116.47,122.56,8.27,29,116.44,122.32,8.8,20,116.53,120.58,57,98.2,101.73,7,50-75,71,2017-02-17 00:00:00,55,80-90,NaN,24,75-90,NaN,103,30,NaN,15,NaN,NaN,I_481,2017-02-17 00:00:00,NaN,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,B,8,NaN,6,NaN,12,3,5,NaN,NaN,NaN,217201772669916,2017-02-17 00:00:00,104,98,106,93,101,NaN,NaN,NaN,Complete
11,14,FU Visit 6,NaN,NaN,2017-09-15 00:00:00,NaN,84695,hfa,120.9,130.529,134.17,2111,105.981,113.037,2241,4,6,9,4,11,6,50-75,NaN,NaN,NaN,NaN,NaN,NaN,118.24,124.23,8.42,30,113.82,119.68,8.67,20,120.2,124.32,59,98.2,101.73,7,50-75,63,2017-09-15 00:00:00,40,40-50,NaN,18,25-50,NaN,84,50,NaN,15,NaN,NaN,I_597,2017-09-15 00:00:00,unscorable,NaN,10,5,5,3,5,3,9,2,1,3,1,1,1,30,A,11,NaN,11,NaN,NaN,NaN,NaN,12,5,11,217201772669916,2017-09-15 00:00:00,102,98,105,93,103,NaN,NaN,NaN,Complete
12,14,FU Visit 7,NaN,NaN,2018-04-16 00:00:00,NaN,12836,wtc,132.08,135.857,141.764,2189,114.399,120.015,2370,5,7,10,6,10,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,119.76,125.35,8.56,30,108.67,114.54,8.41,20,125.84,129.77,62,98.2,101.73,7,50-75,61,2018-04-16 00:00:00,51,70-80,NaN,17,25-50,NaN,113,<10,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,9,4,5,3,5,2,9,2,1,3,1,1,1,28,B,10,NaN,9,NaN,12,5,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
29,26,Baseline,"When I was very young, living in the Astoria n...",I'm a journalist who has covered the rise of t...,2015-02-03 00:00:00,RAVLT rec not administered,36080,NaN,148.32,145.587,151.803,2387,123.836,129.301,2536,5,9,12,3,NaN,NaN,>75,98.49,99.03,441.15,117.08,123.29,20,90.58,96.61,7.23,30,80.58,84.76,7.13,20,96.19,99.72,50,NaN,NaN,NaN,<25,56,NaN,53,80,NaN,26,>90,NaN

Column index Dict

Investigate score spread

In [28]:
datasheet.iloc[tech_ix,51:54]

,51,52,53
6,NaN,NaN,NaN
29,NaN,NaN,NaN
51,84,50,NaN
56,72,20-30,NaN
78,66,60-70,NaN
89,35,>90,NaN
100,74,20-30,NaN
107,NaN,NaN,NaN
118,55,80-90,NaN
154,45,80-90,NaN


In [29]:
#MMSE
datasheet.iloc[tech_all_ix][[0,1,4,74]]

,0,1,4,74
6,14,Baseline,2015-02-26 00:00:00,30
7,14,FU Visit 2,2015-07-14 00:00:00,30
8,14,FU Visit 3,2016-03-07 00:00:00,29
9,14,FU Visit 4,2016-08-25 00:00:00,30
10,14,FU Visit 5,2017-02-17 00:00:00,30
11,14,FU Visit 6,2017-09-15 00:00:00,30
12,14,FU Visit 7,2018-04-16 00:00:00,28
29,26,Baseline,2015-02-03 00:00:00,30
30,26,FU Visit 2,2015-07-28 00:00:00,30
31,26,FU Visit 3,2016-01-28 00:00:00,29


In [30]:
#MMSE scores
#6 unique...
np.unique([int(i) for i in datasheet.iloc[tech_all_ix][74].dropna()],return_counts=True)

(array([23, 26, 27, 28, 29, 30]),
 array([  1,   2,   1,  16,  48, 152], dtype=int64))

In [31]:
#Min MMSE
datasheet.iloc[tech_all_ix][74].min()

23

In [32]:
#The maximum MMSE score is 30 points. A score of 20 to 24 suggests mild dementia, 13 to 20 suggests moderate dementia, and less than 12 indicates severe dementia.
#participants with MCI...one ---
datasheet.iloc[tech_all_ix][74][datasheet.iloc[tech_all_ix][74] < 25]

390    23
Name: 74, dtype: object

In [33]:
#Unique words

In [34]:
len(tech_ix)

50

## Lancet dataset

In [35]:
lancet = pd.read_excel('Lancet.xlsx',header=None)

In [36]:
lancet.head(50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402
0,Record ID,Event Name,Gender,Reason for appointment,Other,Calculated Age,Has someone in your family ever been diagnosed...,Were you or anyone in your close family ever d...,Are you having any symptoms of memory loss or ...,Do you think the patient could undertake one h...,Physician,"Are you coming from out of town, or do you hav...",Type of visit?,Date of first visit?,Date of Consent,Complete?,Clinical Diagnosis (RSI),Clinical Diagnosis (RSI_Abbreviated),Doctor-Rated Compliance (0-10),Date of Neurobehavioral Status Exam,Notes,NIH Toolbox ID,NIH Toolbox Password,Crystallized Composite fully adjusted,Picture Vocab fully adjusted,Picture Vocab age adjusted,Picture Vocab computed,Oral Reading Recognition fully adjusted,Oral Reading Recognition age adjusted,Oral Reading Recognition computed,RAVLT Trial1,RAVLT Trial2,RAVLT Trial3,RAVLT Delayed Recall,RAVLT: Total Recognition Score,RAVLT: False Positive Score,RAVLT Percentile,Picture Sequence Memory fully adjusted,Picture Sequence Age Adjusted,Picture Sequence Memory computed score,List Sorting fully adjusted,List Sorting Age Adjusted,List Sorting raw score,Dimensional Change Card Sort Fully Adjusted,Dimensional Change Age Adjusted,Dimensional Change computed score,Dimensional Change raw score,Flanker Inhibitory Attention & Control Fully A...,Flanker Inhibitory Attention & Control age adj...,Flanker computed score,...,LBM% of Trunk,LBM of Right Leg,LBM% of Right Leg,LBM of Left Leg,LBM% of Left Leg,Leg Lean Mass,TBW of Right Arm,TBW of Left Arm,TBW of Trunk,TBW of Right Leg,TBW of Left Leg,ICW of Right Arm,ICW of Left Arm,ICW of Trunk,ICW of Right Leg,ICW of Left Leg,ECW of Right Arm,ECW of Left Arm,ECW of Trunk,ECW of Right Leg,ECW of Left Leg,ECW/TBW,ECW/TBW of Right Arm,ECW/TBW of Left Arm,ECW/TBW of Trunk,ECW/TBW of Right Leg,ECW/TBW of Left Leg,BFM of Right Arm,BFM% of Right Arm,BFM of Left Arm,BFM% of Left Arm,BFM of Trunk,BFM% of Trunk,BFM of Right Leg,BFM% of Right Leg,BFM of Left Leg,BFM% of Left Leg,BFM Control,LBM Control,BMR (Basal Metabolic Rate),VFL (Visceral Fat Level),VFA (Visceral Fat Area),AC (Arm Circumference),TBW/LBM,FFM,%FFM,DEE,TBW (hidden),BMR (hidden),Complete?
1,10,Baseline,Male,Memory Loss Treatment,NaN,82,NaN,NaN,Yes,NaN,Seifan,NaN,Follow-Up Visit,NaN,NaN,Complete,Amnestic MCI-Multiple Domain,MCI,8,2015-02-23 00:00:00,"RAVLT recognition, Odor ID and Trails B not ad...",85176,NaN,83.32,73.6571,83.1442,1464,100.106,109.371,2180,2,3,3,0,NaN,NaN,<25,95.04,95.98,407.08,68.03,74.62,8,82.9,88.57,5.85,30,68.51,72.43,4.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete
2,10,FU Visit 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Incomplete
3,10,FU Visit 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-28 00:00:00,Trails B incomplete,65967,NaN,129.69,104.74,107.1,1739,127.399,132.224,2578,3,3,4,0,9,16,<25,NaN,NaN,NaN,NaN,NaN,NaN,81.81,87.53,5.74,25,81.03,86.93,6.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1708.53,NaN,NaN,NaN,NaN,124.25,0.755454,2733.65,42.5636,NaN,Complete
4,13,Basel

In [37]:
[(ix,name) for (ix,name) in enumerate(lancet.iloc[0])]

[(0, 'Record ID'),
 (1, 'Event Name'),
 (2, 'Gender'),
 (3, 'Reason for appointment'),
 (4, 'Other'),
 (5, 'Calculated Age'),
 (6,
  "Has someone in your family ever been diagnosed with Alzheimer's Disease?"),
 (7,
  'Were you or anyone in your close family ever diagnosed by an education specialist with a learning disability or ADHD?'),
 (8, 'Are you having any symptoms of memory loss or cognitive impairment?'),
 (9,
  'Do you think the patient could undertake one hour of cognitive testing?'),
 (10, 'Physician'),
 (11,
  'Are you coming from out of town, or do you have a lengthy travel time to the clinic?'),
 (12, 'Type of visit?'),
 (13, 'Date of first visit?'),
 (14, 'Date of Consent'),
 (15, 'Complete?'),
 (16, 'Clinical Diagnosis (RSI)'),
 (17, 'Clinical Diagnosis (RSI_Abbreviated) '),
 (18, 'Doctor-Rated Compliance (0-10)'),
 (19, 'Date of Neurobehavioral Status Exam'),
 (20, 'Notes'),
 (21, 'NIH Toolbox ID'),
 (22, 'NIH Toolbox Password'),
 (23, 'Crystallized Composite fully adju

In [38]:
lancet_id = set(lancet[0][1:])

In [39]:
[no_lance for no_lance in tech_id if no_lance not in lancet_id]

[]

In [40]:
lancet.index

RangeIndex(start=0, stop=635, step=1)

In [41]:
lancet_all_ix = [ix for ix in lancet.index if lancet.loc[ix][0] in tech_id]

In [42]:
lancet_id_ix = lancet.iloc[lancet_all_ix,[0,17]]

In [43]:
lancet_id_ix = lancet_id_ix.dropna()

In [44]:
lancet_id_ix

,0,17
7,14,MCI
34,26,Preclinical PD/DLB
63,64,SCI
70,67,NL
101,101,SCI
112,110,NL
125,135,NL
133,139,NL
146,145,NL
192,205,NL


In [193]:
lancet_id_ix[17].value_counts()

NL                    34
SCI                    9
MCI                    4
Preclinical AD         2
Preclinical PD/DLB     1
Name: 17, dtype: int64

In [45]:
len(lancet_id_ix) == len(ix_id_series)

True

In [46]:
lancet_target = [0 if record_id=='NL' else 1 for record_id in lancet_id_ix[17]]

In [47]:
lancet_target = pd.Series(lancet_target,index=lancet_id_ix[0])

In [199]:
lancet_target_sci_ct = [0 if record_id=='NL' or record_id =='SCI' else 1 for record_id in lancet_id_ix[17]]

In [201]:
lancet_target_sci_ct = pd.Series(lancet_target_sci_ct,index=lancet_id_ix[0])

## Metadata

Gender

In [48]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Female')[0])

35

In [49]:
len(np.where(lancet.iloc[lancet_all_ix][2] == 'Male')[0])

15

Age

In [50]:
lancet.iloc[lancet_all_ix][5].describe()

count     50
unique    30
top       58
freq       3
Name: 5, dtype: int64

In [51]:
mean(lancet.iloc[lancet_all_ix][5].dropna())

58.22

In [52]:
max(lancet.iloc[lancet_all_ix][5].dropna())

78

In [53]:
min(lancet.iloc[lancet_all_ix][5].dropna())

27

In [54]:
lancet.iloc[520:530]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402
520,559,Baseline,Female,Memory Loss Prevention,NaN,72,Yes,No,No,NaN,Seifan,No,New patient Visit,NaN,NaN,Complete,SCI,SCI,4,2016-08-01 00:00:00,NaN,32181,yet,124.37,132.754,135.907,2136,110.115,116.308,2304,5,13,15,13,15,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,107.25,114.31,7.61,30,113.13,119.16,8.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1424.8,NaN,NaN,NaN,NaN,94.14,0.716761,2137.3,27.4323,NaN,Complete
521,559,FU Visit 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-06 00:00:00,NaN,97232,rrz,132.55,137.606,145.452,2220,112.549,118.316,2336,4,7,7,6,13,4,50-75,NaN,NaN,NaN,NaN,NaN,NaN,113.69,120.02,8.04,29,114.11,120.13,8.69,...,100.2,14,91.4,13.91,90.9,27.9,3.64,4.21,34.4,10.96,10.87,2.25,2.6,20.9,6.66,6.64,1.39,1.61,13.4,4.3,4.23,0.39,0.383,0.38,0.39,0.393,0.39,2.6,113.6,2.2,96.8,19.2,151.6,4.9,85.9,4.9,86.3,-5.3,6.8,1314,level 8,81.8,11.4,73.4,NaN,NaN,NaN,NaN,NaN,Complete
522,559,FU Visit 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-28 00:00:00,NaN,64007,qdw,134.24,137.698,145.501,2222,114.952,120.638,2378,6,9,12,10,15,2,>75,NaN,NaN,NaN,NaN,NaN,NaN,108.48,115.25,7.68,28,114.36,120.15,8.7,...,97.6,14.31,93.4,14.31,93.3,28.6,3.48,4.08,33.5,11.2,11.2,2.14,2.54,20.3,6.77,6.77,1.34,1.54,13.2,4.43,4.43,0.392,0.384,0.381,0.391,0.396,0.396,2.6,116.9,2.2,99.5,18.7,149.9,5.3,91.2,5.3,91.2,-5.7,6.2,1313,level 8,80.7,11.4,73.5,NaN,NaN,NaN,NaN,NaN,Complete
523,559,FU Visit 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-06 00:00:00,NaN,47397,unn,123.28,132.978,136.117,2138,108.504,114.961,2280,5,7,8,6,13,2,50-75,NaN,NaN,NaN,NaN,NaN,NaN,111.84,118.3,7.9,29,109.29,115.34,8.45,...,97.7,13.96,90.6,13.82,89.7,27.8,3.57,4.03,33.7,10.94,10.82,2.2,2.49,20.5,6.64,6.57,1.37,1.54,13.2,4.3,4.26,0.391,0.384,0.38,0.391,0.394,0.394,2.9,126.2,2.4,110.6,20.5,164.3,5.3,94.1,5.3,93.8,-8.4,7.5,1300,NaN,87.1,NaN,73.5,NaN,NaN,NaN,NaN,NaN,Complete
524,567,Baseline,Female,Memory Loss Prevention,NaN,27,Yes,No,No,NaN,Isaacson,No,New patient Visit,NaN,NaN,Complete,NL,NL,0,2016-01-29 00:00:00,"logical memory scored out of 50, not included",93701,saw,147.07,142.36,149.182,2190,127.502,131.71,2533,8,10,12,10,14,1,>75,NaN,NaN,NaN,NaN,NaN,NaN,101.39,108.38,9.18,30,96.7,101.52,9.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1412.27,NaN,NaN,NaN,NaN,92.69,NaN,2118.4,NaN,NaN,Complete
525,567,FU Visit 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-21 00:00:00,NaN,54570,kre,149.67,145.368,147.899,2179,129.998,131.635,2529,8,12,13,13,14,0,>75,NaN,NaN,NaN,NaN,NaN,NaN,108.8,107.22,9.11,29,102.7,101.07,9.06,...,95.7,15.7,99.3,15.92,100.7,31.6,3.73,3.7,33.7,12.21,12.41,2.34,2.29,20.9,7.56,7.67,1.39,1.41,12.8,4.65,4.74,0.381,0.375,0.378,0.381,0.381,0.383,3.5,158.7,3.5,161.5,24.9,201.9,7.9,140.5,7.9,140.5,-20.1,1.8,1344,level 10,103.5,11.9,73.1,NaN,NaN,NaN,NaN,NaN,Complete
526,568,Baseline,Male,Memory Loss Prevention,NaN,61,Yes,No,No,NaN,Isaacson,No,Follow-Up Visit,2016-01-11 00:00:00,2016-03-30 00:00:00,Complete,NL,NL,9,2016-01-21 00:00:00,NaN,70423,ter,132.22,129.528,134.212,2128,121.586,128.817,2489,4,6,7,3,10,5,<25,NaN,NaN,NaN,NaN,NaN,NaN,113.86,120.58,9.2,29,105.35,114.1,8.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1450.28,NaN,NaN,NaN,NaN,96.74,0.653152,2465.48,31.2367,NaN,Comp

Diagnosis

In [55]:
lancet.iloc[lancet_all_ix][[16,17]].dropna()

,16,17
7,MCI-NOS,MCI
34,Preclinical PD/DLB,Preclinical PD/DLB
63,SCI,SCI
70,NL,NL
101,SCI,SCI
112,NL,NL
125,NL,NL
133,NL,NL
146,NL,NL
192,NL,NL


In [57]:
np.where(min(lancet.iloc[lancet_all_ix][5].dropna()))

(array([0], dtype=int64),)

# Reference

## Column names

In [58]:
[(i,test) for( i,test) in enumerate(datasheet.iloc[0])]

[(0, 'Record ID'),
 (1, 'Event Name'),
 (2,
  'Write a short sketch about a memory from your childhood and why it is memorable or important to you. Feel free to discuss an event with family or friends, a place you traveled, or a significant time in your life.  Please limit your response to no more than 1-2 paragraphs.  '),
 (3,
  'How does technology and social media impact the daily lives of you and your family?Please limit your response to no more than 1-2 paragraphs.  '),
 (4, 'Date of Neurobehavioral Status Exam'),
 (5, 'Notes'),
 (6, 'NIH Toolbox ID'),
 (7, 'NIH Toolbox Password'),
 (8, 'Crystallized Composite fully adjusted'),
 (9, 'Picture Vocab fully adjusted'),
 (10, 'Picture Vocab age adjusted'),
 (11, 'Picture Vocab computed'),
 (12, 'Oral Reading Recognition fully adjusted'),
 (13, 'Oral Reading Recognition age adjusted'),
 (14, 'Oral Reading Recognition computed'),
 (15, 'RAVLT Trial1'),
 (16, 'RAVLT Trial2'),
 (17, 'RAVLT Trial3'),
 (18, 'RAVLT Delayed Recall'),
 (19, 'RA

## 466k Word list

In [61]:
with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]

In [62]:
len(word_list)

466544

## nltk pos data dictionary

In [63]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

# Feature Extraction

In [64]:
#memory response
mem_corpus = datasheet.iloc[memory_ix,2]

In [65]:
#tech response
tech_corpus = datasheet.iloc[tech_ix,3]

# Helper Functions

### try-catch comprehension hack
https://stackoverflow.com/questions/1528237/how-can-i-handle-exceptions-in-a-list-comprehension-in-python

In [66]:
def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    
    except ZeroDivisionError as z:
        return None
    except statistics.StatisticsError:
        print('Stat error at ix')
        return handle(e)
    except Exception as e:
        return handle(e)

## Tokenization

In [67]:
#Mis-spellings, word length
#rm puctuation
#keep contractions, posessives
def list_tokenize(response):
    return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()

## Information extraction nltk

In [68]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

## Uniq, V

In [69]:
def uniq_words (response,token_list=None, counts_vector_return=False,uniq_set_return=False):
    if token_list is None: token_list = list_tokenize(response)
    if counts_vector_return and uniq_set_return: return np.unique(token_list,return_counts=True)
    
    if counts_vector_return and not uniq_set_return: return np.unique(token_list,return_counts=True)[1]
    
    if uniq_set_return and not counts_vector_return: return np.unique(token_list)
    #typical case, investigating number of uniq responses
    
    return len(np.unique(token_list))
        

## V1 

In [70]:
def single_appearances (response,counts=None):
    if counts is None: counts = uniq_words(response,counts_vector_return=True)
    return len(np.where(counts==1)[0])

## N-grams

In [71]:
#http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
#input
def find_bigrams(response, return_list=False):
    input_list = list_tokenize(response)
    bigrams = zip(input_list, input_list[1:])
    if return_list: return [b for b in bigrams]
    else: return bigrams

## Word count

In [72]:
def word_count(response):
    return len(list_tokenize(response))

## Sentence count

In [73]:
def sentence_count(response):
    return len(ie_preprocess(response))

### POS tagging

In [74]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def ie_preprocess(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    return sentences

## Rates

In [75]:
def pos_rate(response,pos_response=None,tag_set=None,denom_set=None,N=None,nn=False,pn=False,vb=False,proper=None):
    
    if pos_response is None: pos_response = ie_preprocess(response)
        
    
    if tag_set is None:
        #common nouns, excluding proper nouns
        if nn: tag_set = ['NN','NNS']
        elif vb: tag_set = ['VB','VBD','VBG','VBN','VBP','VBZ'] 
        elif pn: tag_set = ['PRP','PRP$']
        elif proper: 
            tag_set = ['PRP','PRP$','WP','WP$']
            denom_set = ['PRP','PRP$','WP','WP$','NN','NNP','NNPS','NNS']
        
    pos_count = sum([1 for sentence in pos_response for word in sentence if word[1] in tag_set])
    
    if denom_set: 
        denom_count = sum([1 for sentence in pos_response for word in sentence if word[1] in denom_set])
    else:
        if N is None: N = word_count(response)
        denom_count = N
        
    return pos_count/denom_count
        

#         sum([1 for sentence in pos_test for word in sentence if word[1] in ['VB','VBD','VBG','VBN','VBP','VBZ'] ])

## Honore's R 
![image.png](attachment:image.png)
https://books.google.com/books?id=CwC4CwAAQBAJ&pg=PA111&lpg=PA111&dq=honore+statistic+lexical+richness&source=bl&ots=LPnM2j9oX5&sig=HGObkoMYRy6lLsgc80Y7tUMT7vk&hl=en&sa=X&ved=2ahUKEwjv0ubE7vTdAhXwYd8KHVRfCiMQ6AEwCXoECAUQAQ#v=onepage&q=honore&f=false

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.2.1359&rep=rep1&type=pdf

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.484.4648&rep=rep1&type=pdf

viii) Honoré’s R statistic.
If V1 denotes the number of hapax legomena recorded, then Honoré (1979) defined his
statistic as:

*R = 100log(N)/(1 - V1/V)*

As a measure of vocabulary richness which has the virtue of being insensitive to text
length, it has been used in stylometric studies by Holmes (1992) and Holmes and
Stylometric analysis of conversational speech
Forsyth (1995). Values of R typically range from 1000 to 2000 with higher values
implying richer vocabularies in the sense that a greater number of words appear
infrequently.


https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf


Assuming log == base e...fits with 1000-2000 assumption




In [76]:
def honore_r(response,v=None,v1=None, N=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_responses(response)
    if v1 is None: 
        counts = uniq_responses(response,counts_vector_return=True)
        v1 = len(np.where(counts==1)[0])
    return (100*math.log(N))/(1-v1/v)

## Brunet's W
W = N**(V −0.165)

(vii) Brunet’s W index.
This index, devised by Brunet (1978) and used successfully by Holmes and Forsyth
(1995), is a measure of vocabulary richness which is insensitive to text length. It is
defined as:

W= N^(V-0.165)
where N is the text length, V the number of different words and (-0.165) is a scaling
constant proposed by Brunet. The measure generally varies between 10 and 20 with a
low value indicating a lexically richer speech.
https://pdfs.semanticscholar.org/11f9/ef33ad001f7638ba29ee8109077de92eb1bb.pdf

In [77]:
def brunet_w(response,N=None, v=None):
    if N is None: N = word_count(response)
    if v is None: v = uniq_responses(response)
    return N**(v-0.165)

## Moving window

In [78]:
#10-window size generator
#https://docs.python.org/release/2.3.5/lib/itertools-example.html
def window(seq, n=10):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [79]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]),8)]

[('This', 'is', 'too', 'much', 'pressure', 'for', 'a', 'writer')]

|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|

# Held-out test set

np.random.choice(tech_ix,(1,int(.20*len(tech_ix))),replace=False)

array([[ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375]], dtype=int64)

In [80]:
held_out_test_ix = np.array([ 29, 464, 324, 277, 242, 496, 107, 359, 411, 375])

In [81]:
#held out test-set coincidently has both tech + mem responses
#dev set has one tech_x responose
tech_x_ix

[234, 280, 337, 519]

In [82]:
train_dev_ix = [t for t in tech_ix if t not in held_out_test_ix]

np.random.choice(tech_ix_train_dev,(1,int(.20*len(tech_ix_train_dev))),replace=False)

## Dev set

tech_ix_dev = np.random.choice(tech_ix_train_dev)

In [83]:
dev_ix = np.array([460,  78, 234, 352, 347, 398, 448, 537])

## Train set

In [84]:
train_ix = [i for i in train_dev_ix if i not in dev_ix]

In [85]:
len(train_ix)

32

In [86]:
train_ix_mem = [m for m in train_ix if m in memory_ix]

In [87]:
len(train_ix_mem)

29

# Features

## Response length

### Response word count

In [88]:
mem_n = pd.Series([word_count(r) for r in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [89]:
mean_mem_length = mean(mem_n)

In [90]:
mean_mem_length

107

In [91]:
tech_x_proxy_length = pd.Series([mean_mem_length for t in tech_x_ix],index=tech_x_ix)

In [92]:
tech_x_proxy_length

234    107
280    107
337    107
519    107
dtype: int64

mem_n = mem_n.append(tech_x_proxy_length)

#tech ix contains all respondent indices
len(mem_n) == len(tech_ix)

In [93]:
tech_n = pd.Series([word_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

#And here you have it
#Merged response lengths
words_count = pd.Series([mem_n[i]+tech_n[i] for i in tech_ix],index=tech_ix)

#Merged length
words_count

### Response sentence count

In [94]:
mem_sentence = pd.Series([sentence_count(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

mean_mem_sentence = mean(mem_sentence)

tech_x_proxy_sentence = pd.Series([mean_mem_sentence for t in tech_x_ix],index=tech_x_ix)

mem_sentence = mem_sentence.append(tech_x_proxy_sentence)

len(mem_sentence) == len(tech_ix)

In [95]:
tech_sentence = pd.Series([sentence_count(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

In [96]:
#And here you have it
#Merged response lengths
sentences_count = pd.Series([mem_sentence[i]+tech_sentence[i] for i in tech_ix],index=tech_ix)

KeyError: 234

In [ ]:
sentences_count.describe()

## Lexical Richness

## V

In [98]:
mem_v = pd.Series([uniq_words(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [99]:
tech_v = pd.Series([uniq_words(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## V1

In [100]:
mem_v1 = pd.Series([single_appearances(mr) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [101]:
tech_v1 = pd.Series([single_appearances(tr) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## MATTR (window size 10) 

In [102]:
window_size = 10

In [103]:
import statistics

In [104]:
[w for w in window(list_tokenize(datasheet.iloc[496,2]))]

[]

In [105]:
list_tokenize(datasheet.iloc[496,2])

['This', 'is', 'too', 'much', 'pressure', 'for', 'a', 'writer']

In [106]:
mem_mattr = []
for ix,mr in zip(memory_ix,datasheet.iloc[memory_ix,2]):
    stripped_mr = list_tokenize(mr)
    if len(stripped_mr) < 10: window_size = len(stripped_mr)
    else: window_size = 10
        
    try:
        mem_mattr.append(mean([len(np.unique(mw))/window_size for mw in window(list_tokenize(mr),window_size)]))
    except statistics.StatisticsError:
        print('Stat error at ix',ix, mr)
mem_mattr_a = pd.Series(mem_mattr,index=memory_ix)

In [107]:
#ratio uniq words in each 10-word window
#avg for all ratios
mem_mattr = pd.Series([mean([len(np.unique(mw))/min(len(list_tokenize(mr)),10) for mw in window(list_tokenize(mr),n=min(len(list_tokenize(mr)),10))]) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [108]:
mem_mattr_a.equals(mem_mattr)

True

In [109]:
tech_mattr = pd.Series([mean([len(np.unique(tw))/min(len(list_tokenize(tr)),10) for tw in window(list_tokenize(tr),n=min(len(list_tokenize(tr)),10))]) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

for tokenized list:

In [110]:
#ten_window = window(stripped_test)

In [111]:
#ratio uniq words in each 10-word window
#avg for all ratios
#mean([len(np.unique(t))/len(t) for t in window(stripped_test)])

#

## Rates

Noun

In [112]:
mem_rate_nn = pd.Series([pos_rate(mr,nn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [113]:
tech_rate_nn = pd.Series([pos_rate(tr,nn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Verb

In [114]:
mem_rate_vb = pd.Series([pos_rate(mr,vb=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [115]:
tech_rate_vb = pd.Series([pos_rate(tr,vb=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

Pronoun

In [116]:
mem_rate_pn = pd.Series([pos_rate(mr,pn=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [117]:
tech_rate_pn = pd.Series([pos_rate(tr,pn=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix) 

Proper noun

In [118]:
mem_rate_proper = pd.Series([pos_rate(mr,proper=True) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [119]:
tech_rate_proper = pd.Series([pos_rate(tr,proper=True) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Initiating with Coordinating Conjunction
Counts, 0/1, rate-per-sentences?

In [120]:
def coord_conjunctions_init(response):
    return sum([1 for sentence in ie_preprocess(response) if sentence[0][1] == 'CC'])

In [121]:
mem_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [122]:
tech_cc_init = pd.Series([catch(lambda:coord_conjunctions_init(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

## Misspellings, words not found in dictionary

In [123]:
def misspelt(response):
    return sum([1 for word in list_tokenize(response) if word.lower() not in word_list])

In [124]:
mem_misspell = pd.Series([catch(lambda:misspelt(mr)) for mr in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [125]:
tech_misspell = pd.Series([catch(lambda:misspelt(tr)) for tr in datasheet.iloc[tech_ix,3]],index=tech_ix)

# Feature Matrix

In [126]:
mem_feature_list = [mem_n,mem_sentence,mem_v,mem_v1,mem_mattr,mem_rate_nn,mem_rate_vb,mem_rate_pn,mem_rate_proper,mem_cc_init,mem_misspell]

In [127]:
mem_feature_df = pd.DataFrame(mem_feature_list).transpose()

In [128]:
mem_feature_df

,0,1,2,3,4,5,6,7,8,9,10
6,80.0,6.0,49.0,31.0,0.956338,0.187500,0.225000,0.150000,0.444444,0.0,0.0
29,101.0,6.0,73.0,60.0,0.932609,0.217822,0.178218,0.069307,0.205882,0.0,1.0
51,142.0,11.0,94.0,74.0,0.972932,0.183099,0.183099,0.169014,0.423729,0.0,4.0
56,156.0,7.0,97.0,73.0,0.946259,0.198718,0.160256,0.141026,0.360656,0.0,6.0
78,72.0,6.0,55.0,44.0,0.976190,0.208333,0.125000,0.125000,0.290323,0.0,3.0
89,209.0,13.0,123.0,90.0,0.965000,0.181818,0.181818,0.143541,0.410256,0.0,6.0
100,65.0,4.0,51.0,43.0,0.975000,0.261538,0.169231,0.061538,0.190476,0.0,0.0
107,184.0,8.0,111.0,81.0,0.956571,0.217391,0.168478,0.114130,0.344262,0.0,3.0
118,38.0,3.0,32.0,27.0,0.958621,0.184211,0.157895,0.157895,0.461538,0.0,0.0
154,70.0,4.0,54.0,44.0,0.944262,0.214286,0.157143,0.114286,0.347826,0.0,1.0


In [129]:
tech_feature_list = [tech_n,tech_sentence,tech_v,tech_v1,tech_mattr,tech_rate_nn,tech_rate_vb,tech_rate_pn,tech_rate_proper,tech_cc_init,tech_misspell]

In [130]:
tech_feature_df = pd.DataFrame(tech_feature_list).transpose()

In [131]:
tech_feature_df

,0,1,2,3,4,5,6,7,8,9,10
6,10.0,1.0,10.0,10.0,1.000000,0.400000,0.100000,0.100000,0.200000,0.0,0.0
29,71.0,3.0,58.0,52.0,0.958065,0.253521,0.169014,0.056338,0.217391,0.0,2.0
51,74.0,5.0,57.0,50.0,0.972308,0.243243,0.148649,0.189189,0.388889,1.0,3.0
56,56.0,5.0,43.0,32.0,0.968085,0.214286,0.232143,0.089286,0.277778,0.0,1.0
78,30.0,4.0,26.0,24.0,0.961905,0.266667,0.200000,0.200000,0.428571,0.0,0.0
89,21.0,2.0,19.0,17.0,1.000000,0.190476,0.238095,0.190476,0.555556,0.0,0.0
100,39.0,2.0,32.0,25.0,0.976667,0.128205,0.256410,0.128205,0.545455,0.0,0.0
107,85.0,6.0,61.0,46.0,0.980263,0.376471,0.082353,0.070588,0.153846,0.0,3.0
118,39.0,3.0,35.0,31.0,0.990000,0.230769,0.205128,0.102564,0.307692,0.0,0.0
154,57.0,5.0,47.0,38.0,0.987500,0.228070,0.210526,0.087719,0.315789,1.0,2.0


In [132]:
X = mem_feature_df.join(tech_feature_df,how='outer',lsuffix='_mem_features',rsuffix='_tech_features')

In [133]:
X.iloc[:,0].mean()

107.95652173913044

In [134]:
X.iloc[:,1].mean()

6.086956521739131

In [135]:
tech_x_ix

[234, 280, 337, 519]

Replace nan with col means

In [136]:
X = X.fillna(X.mean())

In [137]:
lancet_target

0
14     1
26     1
64     1
67     0
101    1
110    0
135    0
139    0
145    0
205    0
233    0
245    0
254    0
263    1
271    1
276    1
291    0
310    0
320    1
322    1
331    1
342    0
382    0
403    0
417    1
420    0
434    0
458    0
465    1
474    0
483    0
512    0
514    0
521    1
532    0
537    0
559    1
567    0
579    0
583    0
587    0
602    0
603    0
604    1
607    0
609    0
616    0
619    0
626    1
631    0
dtype: int64

In [138]:
ix_id_series

6       14
29      26
51      64
56      67
78     101
89     110
100    135
107    139
118    145
154    205
184    233
202    245
217    254
226    263
234    271
242    276
266    291
277    310
280    320
286    322
297    331
303    342
324    382
337    403
347    417
352    420
359    434
375    458
387    465
398    474
411    483
423    512
433    514
438    521
445    532
448    537
460    559
464    567
470    579
474    583
477    587
496    602
499    603
502    604
506    607
509    609
519    616
522    619
537    626
540    631
dtype: int64

In [139]:
#order target by X.index
#ix_id_series indexed by X.index
#lancet_target.loc indexed by ix_id_series record_id
y = [lancet_target.loc[ix_id_series[ix]] for ix in X.index]

In [150]:
y= pd.Series(y,index=X.index)

In [202]:
y_sci_ct = pd.Series([lancet_target_sci_ct.loc[ix_id_series[ix]] for ix in X.index],index=X.index)

In [140]:
#for each col fill empty with avg

## Init modeling

In [141]:
from sklearn import linear_model

In [157]:
#PULL ONLY TRAIN, DEV INDICES FOR TRAINING
lr = linear_model.LogisticRegression()

In [158]:
np.shape(X.loc[train_ix])

(32, 22)

In [159]:
np.shape(y.loc[train_ix])

(32,)

In [160]:
lr.fit(X.loc[train_ix],y.loc[train_ix])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [161]:
lr_preds = lr.predict(X.loc[dev_ix])

In [204]:
lr_sci = linear_model.LogisticRegression()

In [205]:
lr_sci.fit(X.loc[train_ix],y.loc[train_ix])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [206]:
lr_sci_preds = lr_sci.predict(X.loc[dev_ix])

In [207]:
accuracy_score(y.loc[dev_ix],lr_sci_preds)

0.375

In [162]:
from sklearn.metrics import accuracy_score

In [163]:
lr_preds

array([0, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [164]:
accuracy_score(y.loc[dev_ix],lr_preds)

0.375

In [165]:
from sklearn.svm import SVC

In [167]:
svm = SVC()

In [168]:
svm.fit(X.loc[train_ix],y.loc[train_ix])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [169]:
svm_preds = svm.predict(X.loc[dev_ix])

In [171]:
accuracy_score(y.loc[dev_ix],svm_preds)

0.375

In [172]:
from sklearn.ensemble import RandomForestClassifier

In [173]:
rf = RandomForestClassifier()

In [175]:
rf.fit(X.loc[train_ix],y.loc[train_ix])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [176]:
rf_preds = rf.predict(X.loc[dev_ix])

In [177]:
accuracy_score(y.loc[dev_ix],rf_preds)

0.25

In [178]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [179]:
knn.fit(X.loc[train_ix],y.loc[train_ix])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [183]:
knn_preds = knn.predict(X.loc[dev_ix])

In [184]:
knn_preds

array([1, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [186]:
accuracy_score(knn_preds,y.loc[dev_ix])

0.625

In [187]:
y.loc[dev_ix]

460    1
78     1
234    1
352    0
347    1
398    0
448    0
537    1
dtype: int64

In [195]:
from sklearn.metrics import classification_report

In [198]:
print(classification_report(y.loc[dev_ix],knn_preds))

             precision    recall  f1-score   support

          0       0.50      1.00      0.67         3
          1       1.00      0.40      0.57         5

avg / total       0.81      0.62      0.61         8



In [208]:
knn_sci = KNeighborsClassifier(n_neighbors=3)

In [209]:
knn_sci.fit(X.loc[train_ix],y_sci_ct.loc[train_ix])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [210]:
knn_sci_preds = knn_sci.predict(X.loc[dev_ix])

In [211]:
print(classification_report(y_sci_ct.loc[dev_ix],knn_sci_preds))

             precision    recall  f1-score   support

          0       0.86      0.86      0.86         7
          1       0.00      0.00      0.00         1

avg / total       0.75      0.75      0.75         8



In [212]:
y_sci_ct.loc[dev_ix]

460    0
78     0
234    0
352    0
347    0
398    0
448    0
537    1
dtype: int64

In [213]:
knn_sci_preds

array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

## |------------------------------------------------Testing---------------------------------------------------------|

### Stanford parser

In [ ]:
from nltk.tag import StanfordNERTagger
stanford_classifier = r'C:\Users\Renee\StanfordParser\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
stanford_ner_path = r'C:\Users\Renee\StanfordParser\stanford-ner-2018-02-27\stanford-ner.jar'


In [ ]:
st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

In [ ]:
[st.tag(t) for t in [stripped_test]]

In [ ]:
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer [port?] # run server

### Misspellings- (stemmed) words not in dictionary

In [ ]:
#lacking
# from nltk.corpus import words
# word_list = words.words()

In [ ]:
#https://github.com/dwyl/english-words

with open('words.txt','r') as words:
    word_list = words.read().splitlines()

word_list = [w.lower() for w in word_list]

In [ ]:
len(word_list)

In [ ]:
#as ratio to response length?...leave as total misspellings, length accounted for
test_misspellings = sum([1 for t in stripped_test if t.lower() not in word_list])

In [ ]:
test_misspellings

In [ ]:
'couldn\'t' in word_list

In [ ]:
'Little' in word_list

In [ ]:
'little' in word_list

In [ ]:
test_response

## Proportion of pn: (n + pn + proper noun) 
In attempt to model circumlocution, word finding

In [ ]:
sum([1 for sentence in pos_test for word in sentence if word[1] in ['PRP','PRP$','WP','WP$'] ])/sum([1 for sentence in pos_test for word in sentence if word[1] in ['PRP','PRP$','WP','WP$','NN','NNP','NNPS','NNS'] ])

### Honore's R

In [ ]:
mem_honore = pd.Series([catch(lambda:honore_r(mem)) for mem in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
tech_honore = pd.Series([catch(lambda:honore_r(tech)) for tech in datasheet.iloc[tech_ix,3]],index=tech_ix)

### Brunet's W

In [ ]:
pd.Series([catch(lambda:brunet_w(mem)) for mem in datasheet.iloc[memory_ix,2]],index=memory_ix)

In [ ]:
stop

## Word obscurity

## Syntactic complexity, depth of syntactic parse tree

In [ ]:
from nltk.corpus import treebank

In [ ]:
test_response

In [ ]:
stripped_test

In [ ]:
t = treebank.parsed_sents(test_response)
print(t)

In [ ]:
#stemming
#https://pythonprogramming.net/stemming-nltk-tutorial/
# from nltk.corpus import wordnet

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [ ]:
# os.environ['CLASSPATH'] = “stanford-pos”
# os.environ['STANFORD_MODELS'] = “stanford-pos”
# from nltk.tag import StanfordPOSTagger
# st = StanfordPOSTagger(‘english-bidirectional-distsim.tagger’) 
# st.tag(sent.split())

In [ ]:
st.tag(["we\'re"])

In [ ]:
st.tag(['we'])

In [ ]:
st.tag(['are'])

In [ ]:
nltk.pos_tag(['we\'re'])

In [ ]:
nltk.pos_tag(['we'])

In [ ]:
nltk.pos_tag(['are'])

In [ ]:
stagged_mem = [st.tag(t) for t in tokenized_mem]

In [ ]:
stagged_mem

In [ ]:
#https://nlpforhackers.io/stemmers-vs-lemmatizers/
#https://pythonprogramming.net/lemmatizing-nltk-tutorial/
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [ ]:
lemmatizer.lemmatize('notawording')

In [ ]:
lemmatizer.lemmatize('occurred')

In [ ]:
ps.stem('occurred')

In [ ]:
ps.stem('notaworking')